In [36]:
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.decomposition import NMF

<h1> 1. Get Data </h1>

In [51]:
path  = 'C:/Users/thanh/Documents/CARS/Data/ml-100k/'
df = pd.read_csv(path+ 'u.data', sep = '\t', names=['userID','itemID','rating','timestamp'])
df2= pd.read_csv(path+ 'u.genre', sep = '|',names=['Genres','ID'])
df3=pd.read_csv(path+ 'u.item', sep = '|',encoding = "ISO-8859-1",names=['MovieID','Title','realeasDate','videoReleaseDate','URL','unknown','Action','Adventure', 'Animation',  'Children' , 'Comedy',  'Crime',  'Documentary',  'Drama',  'Fantasy',  'Film-Noir',  'Horror',  'Musical',  'Mystery',  'Romance',  'Sci-Fi',  'Thriller',  'War',  'Western' ])
df = df[:800]
df3 = df3.drop(columns=['Title','realeasDate','videoReleaseDate','URL'])
data = pd.merge(df,df3, left_on='itemID',right_on='MovieID',how='inner').drop(columns='MovieID') #Input for side-info
#number of attributes for feaure "GENRE"
F_genre = len(data.columns)-4
df3 = df3.set_index('MovieID')
zzx = data.groupby(['itemID']).mean().drop(columns=['userID','rating','timestamp']).T
# zzx.columns = zzx.iloc[0]
# zzx = zzx[1:]
rating=data.pivot(index='userID',columns='itemID',values='rating').fillna(0)
rate = rating.reset_index().drop(columns=['userID'])

In [53]:
import numpy as np
import pandas as pd 
from scipy.sparse import coo_matrix
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

class MatrixFactorizationCF:
    def __init__(self, n_factors, lambda_reg, n_iterations, alpha):
        self.n_factors = n_factors
        self.lambda_reg = lambda_reg
        self.n_iterations = n_iterations
        self.alpha = alpha
    
    def fit(self, X, item_features):
        model = NMF(n_components=self.n_factors)
        self.n_users, self.n_items = X.shape
        self.R = coo_matrix(X)
        self.item_features = item_features
        self.U = model.fit_transform(self.R)
        self.V = model.components_
        self.item_feature_weights = np.random.normal(size=(self.n_items, self.item_features.shape[1]))
        self.item_feature_bias = np.random.normal(size=self.n_items)
        
        for i in tqdm(range(self.n_iterations)):
            self.update_user_factors()
            self.update_item_factors()
            self.update_item_feature_weights()
            self.update_item_feature_bias()
            
    def predict(self, X):
        return self.U.dot(self.V.T) + self.item_features.dot(self.item_feature_weights.T) + self.item_feature_bias
        
    def update_user_factors(self):
        VV = self.V.T.dot(self.V)
        for i in range(self.n_users):
            V_i = self.V[self.R.row[self.R.row == i], :]
            A = V_i.T.dot(V_i) + self.lambda_reg * np.eye(self.n_factors)
            b = V_i.T.dot(self.R.data[self.R.row == i]) + self.alpha * self.U[i, :]
            self.U[i, :] = np.linalg.solve(A, b)
    
    def update_item_factors(self):
        UU = self.U.T.dot(self.U)
        for j in range(self.n_items):
            U_j = self.U[self.R.col[self.R.col == j], :]
            A = U_j.T.dot(U_j) + self.lambda_reg * np.eye(self.n_factors)
            b = U_j.T.dot(self.R.data[self.R.col == j]) + self.alpha * (self.V[j, :] + self.item_features[j, :].dot(self.item_feature_weights))
            self.V[j, :] = np.linalg.solve(A, b)
    
    def update_item_feature_weights(self):
        for j in range(self.n_items):
            sim = cosine_similarity(self.item_features[j, :].reshape(1, -1), self.item_features)
            w = sim.dot(self.V) / sim.sum()
            A = w.T.dot(w) + self.lambda_reg * np.eye(self.n_factors)
            b = w.T.dot(self.R.data[self.R.col == j] - self.V[j, :].dot(self.U[self.R.row[self.R.col == j], :].T))
            self.item_feature_weights[j, :] = np.linalg.solve(A, b)
    
    def update_item_feature_bias(self):
        for j in range(self.n_items):
            self.item_feature_bias[j] = (self.R.data[self.R.col == j] - self.U[self.R.row[self.R.col == j], :].dot(self.V[j, :].T) - self.item_features[j, :].dot(self.item_feature_weights.T)).mean()


In [55]:
md = MatrixFactorizationCF(n_factors=10,lambda_reg=0.4,alpha=0.012,n_iterations=250)

